In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import time
import ast
from typing import Type, Any, Callable, Union, List, Optional
from torch import Tensor 
from torchsummary import summary
if torch.cuda.is_available():
    print("Using GPUs")
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

Using GPUs


In [2]:
torch.manual_seed(43)
batch_size = 32

### for CIFAR 10
# stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
## for CIFAR 100
stats = ((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(*stats),
    torchvision.transforms.RandomCrop(32, padding=4, padding_mode='constant'),
    torchvision.transforms.RandomHorizontalFlip(p=0.5)
])

train_set = torchvision.datasets.CIFAR100(root="data", train=True, download=True, transform=transform)
train_size = len(train_set)
test_set = torchvision.datasets.CIFAR100(root="data", train=False, download=True, transform=transform)
test_set, validation_set = torch.utils.data.random_split(test_set, [5000, 5000])
test_size = len(test_set)
validation_size = len(validation_set)


train_loader = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size, num_workers=4, pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size, num_workers=4, pin_memory=True)

data_loaders = {"train": train_loader, "test": test_loader, "validation": validation_loader}
dataset_sizes = {"train": train_size, "test": test_size, "validation": validation_size}
print(dataset_sizes)

Files already downloaded and verified
Files already downloaded and verified
{'train': 50000, 'test': 5000, 'validation': 5000}


In [3]:
### from https://pytorch.org/hub/pytorch_vision_resnet/

class BasicBlock(nn.Module):

    def __init__(self, inplanes, planes, stride=1, down=False):
        super().__init__()
            
        self.conv1 = nn.Conv2d(inplanes, planes, stride=stride, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.downsample = None
        
        if down:
            self.downsample = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride)
        

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
        
        out += identity
        out = self.relu(out)
        
        return out

    

class ResNet(nn.Module):
    def __init__(self, model_n, num_classes: int = 10):
        super().__init__()

        self.residual_layers = nn.ModuleList([])
        self.model_n = model_n

        ### begining layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding='same')
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        
        
        ######## ResNet blocks [16, 32, 64]
        ### first block, 16 channels
        for i in range(self.model_n):
            self.residual_layers.append(BasicBlock(16, 16).to(device))
            
        
        ### second block, 32 channels
        for i in range(self.model_n):
            if i == 0:
                self.residual_layers.append(BasicBlock(16, 32, stride=2, down=True).to(device))
            else:
                self.residual_layers.append(BasicBlock(32, 32).to(device))
                
                
        ### third block, 64 channels
        for i in range(self.model_n):
            if i == 0:
                self.residual_layers.append(BasicBlock(32, 64, stride=2, down=True).to(device))
                self.inplanes = 64
            else:
                self.residual_layers.append(BasicBlock(64, 64).to(device))
        
    
        ### output layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)


    def forward(self, x: Tensor) -> Tensor:

        ### begining layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        ##### ResNet blocks
        for i, layer in enumerate(self.residual_layers):
            x = layer (x)
            
            
        ### output layers
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


In [4]:
#### Train Configurations, based on DSNet and ResNet paper
model_n = 16
epochs = 100
milestones = [int(epochs*0.5), int(epochs*0.75)]
momentum = 0.9
weight_decay = 0.0001
gamma = 0.1
lr = 0.1

model = ResNet(model_n, num_classes=100)
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)

summary(model, (3, 32, 32))
print('Total Number of Parameters:', sum(p.numel() for p in model.parameters()))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,320
       BatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]           2,320
       BatchNorm2d-8           [-1, 16, 32, 32]              32
              ReLU-9           [-1, 16, 32, 32]               0
       BasicBlock-10           [-1, 16, 32, 32]               0
           Conv2d-11           [-1, 16, 32, 32]           2,320
      BatchNorm2d-12           [-1, 16, 32, 32]              32
             ReLU-13           [-1, 16, 32, 32]               0
           Conv2d-14           [-1, 16,

In [5]:
### Train loop + validation/ also test at the end
print("Configuration: ", "model:ResNet(small)", " model_n:", model_n, " batch size:", batch_size, 
      " optimizer:SGD", " lr:", lr, " epochs:", epochs)

all_epoch_loss = {"train": [], "validation": []}
all_epoch_acc = {"train":  [], "validation": []}

print("----------------------------- Train --------------------------------")
for epoch in range(epochs):
    start_time = time.time()
    print("Epoch {}/{}".format(epoch+1, epochs))
    print("-" * 30)
    
    
    epoch_loss = {"train": 0.0, "validation": 0.0}
    epoch_acc = {"train": 0.0, "validation": 0.0}
    
    running_loss = {"train": 0.0, "validation": 0.0}
    running_corrects = {"train": 0, "validation": 0}
    
    for phase in ["train", "validation"]:
        if phase == "train":
            model.train(True)
        else:
            model.train(False)
        
        for data in data_loaders[phase]:
            inputs, labels = data 
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() # clear all gradients
            
            outputs = model(inputs) # batch_size x num_classes
            _, preds = torch.max(outputs.data, 1) # values, indices
            loss = loss_fn(outputs, labels)
            
            if phase == "train":
                loss.backward()  # compute gradients
                optimizer.step() # update weights/biases
               
            running_loss[phase] += loss.data.item() * inputs.size(0)
            running_corrects[phase] += torch.sum(preds == labels.data).item()
        
        all_epoch_loss[phase].append(running_loss[phase] / dataset_sizes[phase])
        all_epoch_acc[phase].append(running_corrects[phase] / dataset_sizes[phase])
        
        epoch_loss[phase] = running_loss[phase] / dataset_sizes[phase]
        epoch_acc[phase] =  running_corrects[phase] / dataset_sizes[phase]

    # Visualize the loss and accuracy values.
    print({
        'time': np.round(time.time()-start_time, 5),
        'train_loss': np.round(epoch_loss["train"], 5),
        'train_acc': np.round(epoch_acc["train"], 5),
        'val_loss': np.round(epoch_loss["validation"], 5),
        'val_acc': np.round(epoch_acc["validation"], 5),
    })
    
    scheduler.step()

    
    
with open('ResNet_16_bs_32_opt_SGD_lr_1_epochs_100_loss.txt', 'w') as f_loss:
    print(all_epoch_loss, file=f_loss)
    
with open('ResNet_16_bs_32_opt_SGD_lr_1_epochs_100_acc.txt', 'w') as f_acc:
    print(all_epoch_acc, file=f_acc)
    
### evaluating the model with test set
print("----------------------------- Test --------------------------------")
with torch.no_grad():
    model.eval()
    running_loss = 0
    running_corrects = 0

    for data in test_loader:
        inputs, labels = data 

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # clear all gradients

        outputs = model(inputs) # batch_size x num_classes
        _, preds = torch.max(outputs.data, 1) # values, indices
        loss = loss_fn(outputs, labels)

        running_loss += loss.data.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    # Visualize the loss and accuracy values.
    print({
    'time': np.round(time.time()-start_time, 5),
    'test_loss': np.round(running_loss/ dataset_sizes['test'], 5),
    'test_acc': np.round(running_corrects/ dataset_sizes['test'], 5),
    })

Configuration:  model:ResNet(small)  model_n: 16  batch size: 32  optimizer:SGD  lr: 0.1  epochs: 100
----------------------------- Train --------------------------------
Epoch 1/100
------------------------------
{'time': 100.46282, 'train_loss': 4.56738, 'train_acc': 0.01744, 'val_loss': 4.43006, 'val_acc': 0.027}
Epoch 2/100
------------------------------
{'time': 95.46393, 'train_loss': 4.17949, 'train_acc': 0.05478, 'val_loss': 3.99101, 'val_acc': 0.0846}
Epoch 3/100
------------------------------
{'time': 96.35463, 'train_loss': 3.8713, 'train_acc': 0.09602, 'val_loss': 3.65268, 'val_acc': 0.137}
Epoch 4/100
------------------------------
{'time': 103.21976, 'train_loss': 3.55425, 'train_acc': 0.14862, 'val_loss': 3.43548, 'val_acc': 0.168}
Epoch 5/100
------------------------------
{'time': 99.14401, 'train_loss': 3.29222, 'train_acc': 0.19906, 'val_loss': 3.10503, 'val_acc': 0.2394}
Epoch 6/100
------------------------------
{'time': 100.02316, 'train_loss': 3.07656, 'train_acc

{'time': 100.66835, 'train_loss': 0.50847, 'train_acc': 0.84142, 'val_loss': 1.66694, 'val_acc': 0.6094}
Epoch 57/100
------------------------------
{'time': 98.94228, 'train_loss': 0.50152, 'train_acc': 0.8436, 'val_loss': 1.65755, 'val_acc': 0.6026}
Epoch 58/100
------------------------------
{'time': 102.47038, 'train_loss': 0.48986, 'train_acc': 0.85046, 'val_loss': 1.67844, 'val_acc': 0.61}
Epoch 59/100
------------------------------
{'time': 95.39317, 'train_loss': 0.48269, 'train_acc': 0.8496, 'val_loss': 1.73498, 'val_acc': 0.6062}
Epoch 60/100
------------------------------
{'time': 95.92352, 'train_loss': 0.47406, 'train_acc': 0.85156, 'val_loss': 1.6815, 'val_acc': 0.6098}
Epoch 61/100
------------------------------
{'time': 100.45637, 'train_loss': 0.46628, 'train_acc': 0.85378, 'val_loss': 1.69536, 'val_acc': 0.6142}
Epoch 62/100
------------------------------
{'time': 100.73071, 'train_loss': 0.45747, 'train_acc': 0.85442, 'val_loss': 1.71739, 'val_acc': 0.6148}
Epoch 63/

In [9]:
s = open('ResNet_16_bs_32_opt_SGD_lr_1_epochs_100_loss.txt', 'r').read()
ast.literal_eval(s)

In [5]:
### Train loop + validation/ also test at the end
print("Configuration: ", "model:ResNet(small)", " model_n:", model_n, " batch size:", batch_size, 
      " optimizer:SGD", " lr:", lr, " epochs:", epochs)

all_epoch_loss = {"train": [], "validation": []}
all_epoch_acc = {"train":  [], "validation": []}

print("----------------------------- Train --------------------------------")
for epoch in range(epochs):
    start_time = time.time()
    print("Epoch {}/{}".format(epoch+1, epochs))
    print("-" * 30)
    
    
    epoch_loss = {"train": 0.0, "validation": 0.0}
    epoch_acc = {"train": 0.0, "validation": 0.0}
    
    running_loss = {"train": 0.0, "validation": 0.0}
    running_corrects = {"train": 0, "validation": 0}
    
    for phase in ["train", "validation"]:
        if phase == "train":
            model.train(True)
        else:
            model.train(False)
        
        for data in data_loaders[phase]:
            inputs, labels = data 
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() # clear all gradients
            
            outputs = model(inputs) # batch_size x num_classes
            _, preds = torch.max(outputs.data, 1) # values, indices
            loss = loss_fn(outputs, labels)
            
            if phase == "train":
                loss.backward()  # compute gradients
                optimizer.step() # update weights/biases
               
            running_loss[phase] += loss.data.item() * inputs.size(0)
            running_corrects[phase] += torch.sum(preds == labels.data).item()
        
        all_epoch_loss[phase].append(running_loss[phase] / dataset_sizes[phase])
        all_epoch_acc[phase].append(running_corrects[phase] / dataset_sizes[phase])
        
        epoch_loss[phase] = running_loss[phase] / dataset_sizes[phase]
        epoch_acc[phase] =  running_corrects[phase] / dataset_sizes[phase]

    # Visualize the loss and accuracy values.
    print({
        'time': np.round(time.time()-start_time, 5),
        'train_loss': np.round(epoch_loss["train"], 5),
        'train_acc': np.round(epoch_acc["train"], 5),
        'val_loss': np.round(epoch_loss["validation"], 5),
        'val_acc': np.round(epoch_acc["validation"], 5),
    })
    
    scheduler.step()

    
    
with open('ResNet_3_bs_32_opt_SGD_lr_1_epochs_100_loss.txt', 'w') as f_loss:
    print(all_epoch_loss, file=f_loss)
    
with open('ResNet_3_bs_32_opt_SGD_lr_1_epochs_100_acc.txt', 'w') as f_acc:
    print(all_epoch_acc, file=f_acc)
    
### evaluating the model with test set
print("----------------------------- Test --------------------------------")
with torch.no_grad():
    model.eval()
    running_loss = 0
    running_corrects = 0

    for data in test_loader:
        inputs, labels = data 

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # clear all gradients

        outputs = model(inputs) # batch_size x num_classes
        _, preds = torch.max(outputs.data, 1) # values, indices
        loss = loss_fn(outputs, labels)

        running_loss += loss.data.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    # Visualize the loss and accuracy values.
    print({
    'time': np.round(time.time()-start_time, 5),
    'test_loss': np.round(running_loss/ dataset_sizes['test'], 5),
    'test_acc': np.round(running_corrects/ dataset_sizes['test'], 5),
    })

Configuration:  model:ResNet(small)  model_n: 3  batch size: 32  optimizer:SGD  lr: 0.1  epochs: 100
----------------------------- Train --------------------------------
Epoch 1/100
------------------------------
{'time': 34.01057, 'train_loss': 4.09702, 'train_acc': 0.06634, 'val_loss': 3.74329, 'val_acc': 0.12}
Epoch 2/100
------------------------------
{'time': 30.53767, 'train_loss': 3.57187, 'train_acc': 0.14674, 'val_loss': 3.32252, 'val_acc': 0.1924}
Epoch 3/100
------------------------------
{'time': 29.13071, 'train_loss': 3.21325, 'train_acc': 0.213, 'val_loss': 3.12238, 'val_acc': 0.2362}
Epoch 4/100
------------------------------
{'time': 29.05568, 'train_loss': 2.93294, 'train_acc': 0.26122, 'val_loss': 2.89824, 'val_acc': 0.2798}
Epoch 5/100
------------------------------
{'time': 31.18892, 'train_loss': 2.69874, 'train_acc': 0.30884, 'val_loss': 2.61728, 'val_acc': 0.3282}
Epoch 6/100
------------------------------
{'time': 29.74719, 'train_loss': 2.50385, 'train_acc': 0

{'time': 29.10951, 'train_loss': 0.88921, 'train_acc': 0.7356, 'val_loss': 1.4694, 'val_acc': 0.6074}
Epoch 57/100
------------------------------
{'time': 28.05627, 'train_loss': 0.87839, 'train_acc': 0.738, 'val_loss': 1.44947, 'val_acc': 0.6162}
Epoch 58/100
------------------------------
{'time': 28.38957, 'train_loss': 0.87642, 'train_acc': 0.73786, 'val_loss': 1.48212, 'val_acc': 0.6104}
Epoch 59/100
------------------------------
{'time': 29.40691, 'train_loss': 0.87187, 'train_acc': 0.7386, 'val_loss': 1.47409, 'val_acc': 0.6156}
Epoch 60/100
------------------------------
{'time': 28.66238, 'train_loss': 0.86628, 'train_acc': 0.74078, 'val_loss': 1.47077, 'val_acc': 0.6084}
Epoch 61/100
------------------------------
{'time': 28.59666, 'train_loss': 0.86225, 'train_acc': 0.74422, 'val_loss': 1.46337, 'val_acc': 0.6198}
Epoch 62/100
------------------------------
{'time': 28.79412, 'train_loss': 0.85252, 'train_acc': 0.74358, 'val_loss': 1.4827, 'val_acc': 0.6144}
Epoch 63/100
-

In [5]:
### Train loop + validation/ also test at the end
print("Configuration: ", "model:ResNet(small)", " model_n:", model_n, " batch size:", batch_size, 
      " optimizer:SGD", " lr:", lr, " epochs:", epochs)

all_epoch_loss = {"train": [], "validation": []}
all_epoch_acc = {"train":  [], "validation": []}

print("----------------------------- Train --------------------------------")
for epoch in range(epochs):
    start_time = time.time()
    print("Epoch {}/{}".format(epoch+1, epochs))
    print("-" * 30)
    
    
    epoch_loss = {"train": 0.0, "validation": 0.0}
    epoch_acc = {"train": 0.0, "validation": 0.0}
    
    running_loss = {"train": 0.0, "validation": 0.0}
    running_corrects = {"train": 0, "validation": 0}
    
    for phase in ["train", "validation"]:
        if phase == "train":
            model.train(True)
        else:
            model.train(False)
        
        for data in data_loaders[phase]:
            inputs, labels = data 
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() # clear all gradients
            
            outputs = model(inputs) # batch_size x num_classes
            _, preds = torch.max(outputs.data, 1) # values, indices
            loss = loss_fn(outputs, labels)
            
            if phase == "train":
                loss.backward()  # compute gradients
                optimizer.step() # update weights/biases
               
            running_loss[phase] += loss.data.item() * inputs.size(0)
            running_corrects[phase] += torch.sum(preds == labels.data).item()
        
        all_epoch_loss[phase].append(running_loss[phase] / dataset_sizes[phase])
        all_epoch_acc[phase].append(running_corrects[phase] / dataset_sizes[phase])
        
        epoch_loss[phase] = running_loss[phase] / dataset_sizes[phase]
        epoch_acc[phase] =  running_corrects[phase] / dataset_sizes[phase]

    # Visualize the loss and accuracy values.
    print({
        'time': np.round(time.time()-start_time, 5),
        'train_loss': np.round(epoch_loss["train"], 5),
        'train_acc': np.round(epoch_acc["train"], 5),
        'val_loss': np.round(epoch_loss["validation"], 5),
        'val_acc': np.round(epoch_acc["validation"], 5),
    })
    
    scheduler.step()

    
    
with open('ResNet_8_bs_32_opt_SGD_lr_1_epochs_100_loss.txt', 'w') as f_loss:
    print(all_epoch_loss, file=f_loss)
    
with open('ResNet_8_bs_32_opt_SGD_lr_1_epochs_100_acc.txt', 'w') as f_acc:
    print(all_epoch_acc, file=f_acc)
    
### evaluating the model with test set
print("----------------------------- Test --------------------------------")
with torch.no_grad():
    model.eval()
    running_loss = 0
    running_corrects = 0

    for data in test_loader:
        inputs, labels = data 

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # clear all gradients

        outputs = model(inputs) # batch_size x num_classes
        _, preds = torch.max(outputs.data, 1) # values, indices
        loss = loss_fn(outputs, labels)

        running_loss += loss.data.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    # Visualize the loss and accuracy values.
    print({
    'time': np.round(time.time()-start_time, 5),
    'test_loss': np.round(running_loss/ dataset_sizes['test'], 5),
    'test_acc': np.round(running_corrects/ dataset_sizes['test'], 5),
    })

Configuration:  model:ResNet(small)  model_n: 8  batch size: 32  optimizer:SGD  lr: 0.1  epochs: 100
----------------------------- Train --------------------------------
Epoch 1/100
------------------------------
{'time': 59.92077, 'train_loss': 4.32535, 'train_acc': 0.0387, 'val_loss': 4.07001, 'val_acc': 0.0672}
Epoch 2/100
------------------------------
{'time': 55.63084, 'train_loss': 3.82518, 'train_acc': 0.10106, 'val_loss': 3.65821, 'val_acc': 0.1388}
Epoch 3/100
------------------------------
{'time': 54.97735, 'train_loss': 3.45851, 'train_acc': 0.16812, 'val_loss': 3.29394, 'val_acc': 0.199}
Epoch 4/100
------------------------------
{'time': 54.96699, 'train_loss': 3.11303, 'train_acc': 0.22886, 'val_loss': 2.93745, 'val_acc': 0.26}
Epoch 5/100
------------------------------
{'time': 55.37949, 'train_loss': 2.82663, 'train_acc': 0.28456, 'val_loss': 2.661, 'val_acc': 0.32}
Epoch 6/100
------------------------------
{'time': 59.1742, 'train_loss': 2.61062, 'train_acc': 0.3263

{'time': 59.08249, 'train_loss': 0.53819, 'train_acc': 0.83212, 'val_loss': 1.45325, 'val_acc': 0.6398}
Epoch 57/100
------------------------------
{'time': 63.60274, 'train_loss': 0.53428, 'train_acc': 0.83316, 'val_loss': 1.47784, 'val_acc': 0.638}
Epoch 58/100
------------------------------
{'time': 62.4783, 'train_loss': 0.51871, 'train_acc': 0.83724, 'val_loss': 1.49112, 'val_acc': 0.6346}
Epoch 59/100
------------------------------
{'time': 62.42422, 'train_loss': 0.51499, 'train_acc': 0.83898, 'val_loss': 1.47314, 'val_acc': 0.6358}
Epoch 60/100
------------------------------
{'time': 61.96302, 'train_loss': 0.50197, 'train_acc': 0.84164, 'val_loss': 1.48724, 'val_acc': 0.645}
Epoch 61/100
------------------------------
{'time': 64.10068, 'train_loss': 0.5001, 'train_acc': 0.84406, 'val_loss': 1.48238, 'val_acc': 0.6414}
Epoch 62/100
------------------------------
{'time': 60.23529, 'train_loss': 0.49169, 'train_acc': 0.84508, 'val_loss': 1.50742, 'val_acc': 0.6334}
Epoch 63/100

In [5]:
#### again 16
### Train loop + validation/ also test at the end
print("Configuration: ", "model:ResNet(small)", " model_n:", model_n, " batch size:", batch_size, 
      " optimizer:SGD", " lr:", lr, " epochs:", epochs)

all_epoch_loss = {"train": [], "validation": []}
all_epoch_acc = {"train":  [], "validation": []}

print("----------------------------- Train --------------------------------")
for epoch in range(epochs):
    start_time = time.time()
    print("Epoch {}/{}".format(epoch+1, epochs))
    print("-" * 30)
    
    
    epoch_loss = {"train": 0.0, "validation": 0.0}
    epoch_acc = {"train": 0.0, "validation": 0.0}
    
    running_loss = {"train": 0.0, "validation": 0.0}
    running_corrects = {"train": 0, "validation": 0}
    
    for phase in ["train", "validation"]:
        if phase == "train":
            model.train(True)
        else:
            model.train(False)
        
        for data in data_loaders[phase]:
            inputs, labels = data 
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad() # clear all gradients
            
            outputs = model(inputs) # batch_size x num_classes
            _, preds = torch.max(outputs.data, 1) # values, indices
            loss = loss_fn(outputs, labels)
            
            if phase == "train":
                loss.backward()  # compute gradients
                optimizer.step() # update weights/biases
               
            running_loss[phase] += loss.data.item() * inputs.size(0)
            running_corrects[phase] += torch.sum(preds == labels.data).item()
        
        all_epoch_loss[phase].append(running_loss[phase] / dataset_sizes[phase])
        all_epoch_acc[phase].append(running_corrects[phase] / dataset_sizes[phase])
        
        epoch_loss[phase] = running_loss[phase] / dataset_sizes[phase]
        epoch_acc[phase] =  running_corrects[phase] / dataset_sizes[phase]

    # Visualize the loss and accuracy values.
    print({
        'time': np.round(time.time()-start_time, 5),
        'train_loss': np.round(epoch_loss["train"], 5),
        'train_acc': np.round(epoch_acc["train"], 5),
        'val_loss': np.round(epoch_loss["validation"], 5),
        'val_acc': np.round(epoch_acc["validation"], 5),
    })
    
    scheduler.step()

    
    
with open('ResNet_16_bs_32_opt_SGD_lr_1_epochs_100_loss.txt', 'w') as f_loss:
    print(all_epoch_loss, file=f_loss)
    
with open('ResNet_16_bs_32_opt_SGD_lr_1_epochs_100_acc.txt', 'w') as f_acc:
    print(all_epoch_acc, file=f_acc)
    
### evaluating the model with test set
print("----------------------------- Test --------------------------------")
with torch.no_grad():
    model.eval()
    running_loss = 0
    running_corrects = 0

    for data in test_loader:
        inputs, labels = data 

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # clear all gradients

        outputs = model(inputs) # batch_size x num_classes
        _, preds = torch.max(outputs.data, 1) # values, indices
        loss = loss_fn(outputs, labels)

        running_loss += loss.data.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    # Visualize the loss and accuracy values.
    print({
    'time': np.round(time.time()-start_time, 5),
    'test_loss': np.round(running_loss/ dataset_sizes['test'], 5),
    'test_acc': np.round(running_corrects/ dataset_sizes['test'], 5),
    })

Configuration:  model:ResNet(small)  model_n: 16  batch size: 32  optimizer:SGD  lr: 0.1  epochs: 100
----------------------------- Train --------------------------------
Epoch 1/100
------------------------------
{'time': 114.81999, 'train_loss': 4.52993, 'train_acc': 0.01976, 'val_loss': 4.44816, 'val_acc': 0.022}
Epoch 2/100
------------------------------
{'time': 110.86615, 'train_loss': 4.09644, 'train_acc': 0.06298, 'val_loss': 3.87522, 'val_acc': 0.0898}
Epoch 3/100
------------------------------
{'time': 115.13954, 'train_loss': 3.75482, 'train_acc': 0.11598, 'val_loss': 3.51611, 'val_acc': 0.1582}
Epoch 4/100
------------------------------
{'time': 108.28486, 'train_loss': 3.4868, 'train_acc': 0.1633, 'val_loss': 3.36987, 'val_acc': 0.1916}
Epoch 5/100
------------------------------
{'time': 107.96477, 'train_loss': 3.26434, 'train_acc': 0.2033, 'val_loss': 3.1193, 'val_acc': 0.233}
Epoch 6/100
------------------------------
{'time': 107.58795, 'train_loss': 3.06294, 'train_ac

{'time': 98.66107, 'train_loss': 0.43433, 'train_acc': 0.86398, 'val_loss': 1.67446, 'val_acc': 0.6266}
Epoch 57/100
------------------------------
{'time': 97.24948, 'train_loss': 0.41584, 'train_acc': 0.86862, 'val_loss': 1.68098, 'val_acc': 0.6208}
Epoch 58/100
------------------------------
{'time': 103.74064, 'train_loss': 0.41192, 'train_acc': 0.87014, 'val_loss': 1.69139, 'val_acc': 0.6194}
Epoch 59/100
------------------------------


KeyboardInterrupt: 